In [1]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from itertools import product
import multiprocessing as mp
from numpy import kron
import os
import warnings
warnings.filterwarnings("ignore")
import sys
path='C:/Users/annie/Spatio-temporal/src/'
if path not in sys.path:
    sys.path.append(path)
from model_st_new import VCM

In [3]:
ycol = 'gmv'#,'cnt_grab', 'cnt_finish']
xcol = 'cnt_call'#,'sum_online_time']
scol = 'cnt_call_1'
acol = 'is_exp'
regcols = ['const'] + [xcol]
adj_mat = np.array([[0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
             [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
             [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
             [1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
             [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
             [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
             [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0],
             [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0],
             [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1],
             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1],
             [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1],
             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]])                                                   
G = 17
adj_mat = adj_mat/np.repeat(adj_mat.sum(axis=0),G).reshape(G,G)

two_sided = False
wild_bootstrap = False
interaction = False

hc = 0.8
IE = True
hc_b = 0.8

### AA test

In [4]:
file = 'V1_CityE_expand_withID_AA.csv'# chengdu from 2019-2-05 to 2019-2-18
df = pd.read_csv('C:/Users/annie/Spatio-temporal/data/'+file, index_col=['grid_id','date','time'])

df['const'] = 1
design = 'st'

M = len(df.index.get_level_values(2).unique())
N = len(df.index.get_level_values(1).unique())
NM = M*N
df[scol] = np.append(np.delete(df[xcol].values *(df.index.get_level_values(2)>0),0),0)

model = VCM(df, ycol, xcol, acol, scol,IE,
                     interaction=interaction,
                     two_sided=two_sided, 
                     wild_bootstrap=wild_bootstrap, 
                    center_x=True, scale_x=True,hc=hc)
model.inference()

df['fitted_DE'] = model.holder['fitted_DE_1'].values
df['resid_DE'] = model.holder['resid_DE_1'].values
df['fitted_IE1'] = model.holder['fitted_IE1_1'].values
df['resid_IE1'] = model.holder['resid_IE1_1'].values
df['fitted_IE2'] = model.holder['fitted_IE2_1'].values
df['resid_IE2'] = model.holder['resid_IE2_1'].values

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", palette="pastel", color_codes=True)
fig = plt.figure(figsize=(14, 4))

plt.subplot(1,2,1)
df.groupby('date')['resid_DE'].plot(use_index=False)

plt.subplot(1,2,2)
plt.scatter(df['gmv'],df['fitted_DE'])

plt.show()

In [ ]:
[model.holder['pvalue_DE'], model.holder['pvalue_IE']]

### AB test

In [ ]:
file = 'V1_CityE_expand_withID_AB.csv'# chengdu from 2019-2-05 to 2019-2-18
df = pd.read_csv('C:/Users/annie/Spatio-temporal/data/'+file, index_col=['grid_id','date','time'])

df['const'] = 1
design = 'st'

M = len(df.index.get_level_values(2).unique())
N = len(df.index.get_level_values(1).unique())
NM = M*N
df[scol] = np.append(np.delete(df[xcol].values *(df.index.get_level_values(2)>0),0),0)

model = VCM(df, ycol, xcol, acol, scol,IE,
                     interaction=interaction,
                     two_sided=two_sided, 
                     wild_bootstrap=wild_bootstrap, 
                    center_x=True, scale_x=True,hc=hc)
model.inference()

df['fitted_DE'] = model.holder['fitted_DE_1'].values
df['resid_DE'] = model.holder['resid_DE_1'].values
df['fitted_IE1'] = model.holder['fitted_IE1_1'].values
df['resid_IE1'] = model.holder['resid_IE1_1'].values
df['fitted_IE2'] = model.holder['fitted_IE2_1'].values
df['resid_IE2'] = model.holder['resid_IE2_1'].values

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", palette="pastel", color_codes=True)
fig = plt.figure(figsize=(14, 4))

plt.subplot(1,2,1)
df.groupby('date')['resid_DE'].plot(use_index=False)

plt.subplot(1,2,2)
plt.scatter(df['gmv'],df['fitted_DE'])

plt.show()

In [ ]:
[model.holder['pvalue_DE'], model.holder['pvalue_IE']]